In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://www.imdb.com/chart/top'

## Fetch the webpage
r = requests.get(url)
print (type(r))

<class 'requests.models.Response'>


In [4]:
r.status_code

200

In [5]:
info = r.text

In [6]:
soup = BeautifulSoup(r.text, 'html.parser')

In [7]:
print (type(soup))

<class 'bs4.BeautifulSoup'>


In [8]:
TITLES = []
YEARS = []
RATINGS = []
URLS = []

ACTORS = []
DIRECTORS = []

In [9]:
table_inf = soup.find_all('tbody', {'class': 'lister-list'})
print (len(table_inf))
print (type(table_inf[0]))

doc = table_inf[0]

1
<class 'bs4.element.Tag'>


In [10]:
rows = doc.find_all('tr')
print (len(rows))

250


In [11]:
test_row = rows[0]
title_column = test_row.find('td', {'class' : 'titleColumn'})
print (title_column.span)
print (title_column.a.text)
print (type(title_column.a.text.encode('utf-8')))

<span class="secondaryInfo">(1994)</span>
The Shawshank Redemption
<class 'bytes'>


In [12]:
count = 0
movie_root_url = 'https://www.imdb.com'
for row in rows:
    #if count>4:
     #   break
    count+=1
    title_column = row.find('td', {'class': 'titleColumn'})
    rating_column = row.find('td', {'class' : 'ratingColumn'})
    
    movie_title = title_column.a.text
    year = title_column.span.text.split('(')[-1].split(')')[0]
    movie_rel_url = title_column.a['href']
    movie_url = movie_root_url + movie_rel_url
    
    imp_people = title_column.a['title']
    director = imp_people.split(',')[0].split('(')[0].strip()
    
    actors = ','.join(imp_people.split(',')[1:]).strip()
    
    rating = rating_column.strong.text
    
    TITLES.append(movie_title.encode('utf-8'))
    YEARS.append(year.encode('utf-8'))
    URLS.append(movie_url.encode('utf-8'))
    RATINGS.append(rating.encode('utf-8'))
    
    ACTORS.append(actors.encode('utf-8'))
    DIRECTORS.append(director.encode('utf-8'))
    
#     print movie_title
#     print year
#     print rating
#     print movie_url
#     print director
#     print actors
    #print '--------'
print (count)

250


In [13]:
print (type(title_column))

<class 'bs4.element.Tag'>


In [14]:
print (len(TITLES))
print (len(DIRECTORS))
print (len(ACTORS))
print (len(RATINGS))

250
250
250
250


In [15]:
empty_df = pd.DataFrame({'Title': TITLES, 'Rating': RATINGS, 'Year': YEARS, 'URL': URLS, 'Director': DIRECTORS, 'Actors': ACTORS}, columns=['Title', 'Year', 'Rating', 'Director', 'Actors', 'URL'])

In [16]:
empty_df.head(n=4)

,Title,Year,Rating,Director,Actors,URL
0,b'The Shawshank Redemption',b'1994',b'9.2',b'Frank Darabont',"b'Tim Robbins, Morgan Freeman'",b'https://www.imdb.com/title/tt0111161/?pf_rd_...
1,b'The Godfather',b'1972',b'9.2',b'Francis Ford Coppola',"b'Marlon Brando, Al Pacino'",b'https://www.imdb.com/title/tt0068646/?pf_rd_...
2,b'The Godfather: Part II',b'1974',b'9.0',b'Francis Ford Coppola',"b'Al Pacino, Robert De Niro'",b'https://www.imdb.com/title/tt0071562/?pf_rd_...
3,b'The Dark Knight',b'2008',b'9.0',b'Christopher Nolan',"b'Christian Bale, Heath Ledger'",b'https://www.imdb.com/title/tt0468569/?pf_rd_...


In [17]:
empty_df.to_csv('./imdb.csv', index=False)

In [18]:
movie_data = pd.read_csv('imdb.csv')
movie_data.head(n=5)

,Title,Year,Rating,Director,Actors,URL
0,b'The Shawshank Redemption',b'1994',b'9.2',b'Frank Darabont',"b'Tim Robbins, Morgan Freeman'",b'https://www.imdb.com/title/tt0111161/?pf_rd_...
1,b'The Godfather',b'1972',b'9.2',b'Francis Ford Coppola',"b'Marlon Brando, Al Pacino'",b'https://www.imdb.com/title/tt0068646/?pf_rd_...
2,b'The Godfather: Part II',b'1974',b'9.0',b'Francis Ford Coppola',"b'Al Pacino, Robert De Niro'",b'https://www.imdb.com/title/tt0071562/?pf_rd_...
3,b'The Dark Knight',b'2008',b'9.0',b'Christopher Nolan',"b'Christian Bale, Heath Ledger'",b'https://www.imdb.com/title/tt0468569/?pf_rd_...
4,b'12 Angry Men',b'1957',b'8.9',b'Sidney Lumet',"b'Henry Fonda, Lee J. Cobb'",b'https://www.imdb.com/title/tt0050083/?pf_rd_...


In [19]:
print (movie_data.values.shape)

(250, 6)


In [20]:
len(URLS)


250

In [22]:
SUMMARY=[]
COLLECTION=[]
GENRE=[]
cnt=0
for i in range(len(URLS)):
    url = URLS[i]
    cnt+=1
    if(cnt%10==0):
        print(cnt)
## Fetch the webpage
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    info_genre= soup.find_all('span', {'class': 'itemprop'})
    genre=info_genre[0].text
    #print(genre)
    #print(info_genre[0].text)
    info_summary= soup.find_all('div', {'class': 'summary_text'})
    summary=info_summary[0].text.strip()
    #print(info_summary[0].text)
    if(soup.find('h4', text='Budget:')):
        info_collection= soup.find('h4', text='Budget:').next_sibling
    else:
        info_collection="NOT AVAILABLE"
    collection=info_collection.strip()
    SUMMARY.append(summary.encode('utf-8'))
    COLLECTION.append(collection.encode('utf-8'))
    GENRE.append(genre.encode('utf-8'))
    

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250


In [23]:
COLLECTION

[b'$25,000,000',
 b'$6,000,000',
 b'$13,000,000',
 b'$185,000,000',
 b'$350,000',
 b'$22,000,000',
 b'$94,000,000',
 b'$8,000,000',
 b'$1,200,000',
 b'$63,000,000',
 b'$93,000,000',
 b'$55,000,000',
 b'$18,000,000',
 b'$160,000,000',
 b'$94,000,000',
 b'$4,400,000',
 b'$25,000,000',
 b'$63,000,000',
 b'$2,000,000',
 b'$321,000,000',
 b'$3,300,000',
 b'$11,000,000',
 b'$33,000,000',
 b'$19,000,000',
 b'$3,180,000',
 b'$20,000,000',
 b'$6,000,000',
 b'$19,000,000',
 b'$70,000,000',
 b'FRF\xc2\xa0115,000,000',
 b'$60,000,000',
 b'$165,000,000',
 b'$7,500,000',
 b'$806,947',
 b'$5,000,000',
 b'$1,500,000',
 b'$950,000',
 b'$1,500,000',
 b'\xe2\x82\xac9,500,000',
 b'$35,000,000',
 b'$90,000,000',
 b'$102,000,000',
 b'$19,000,000',
 b'$18,000,000',
 b'$1,000,000',
 b'$3,300,000',
 b'$103,000,000',
 b'$45,000,000',
 b'$40,000,000',
 b'$9,000,000',
 b'$31,500,000',
 b'$11,000,000',
 b'$2,000,000',
 b'$5,000,000',
 b'$1,752,000',
 b'$3,700,000',
 b'$1,800,000',
 b'$2,000,000',
 b'$935,000',
 b'

In [24]:
print(len(COLLECTION))


250


In [25]:
new_df= pd.DataFrame({'Title': TITLES, 'Rating': RATINGS, 'Year': YEARS, 'URL': URLS, 'Director': DIRECTORS, 'Actors': ACTORS,'Summary':SUMMARY,'Collection':COLLECTION,'Genre':GENRE}, columns=['Title', 'Year', 'Rating', 'Director', 'Actors', 'URL','GENRE','Collection','Summary'])

In [26]:
new_df.head(n=4)

,Title,Year,Rating,Director,Actors,URL,GENRE,Collection,Summary
0,b'The Shawshank Redemption',b'1994',b'9.2',b'Frank Darabont',"b'Tim Robbins, Morgan Freeman'",b'https://www.imdb.com/title/tt0111161/?pf_rd_...,NaN,"b'$25,000,000'",b'Two imprisoned men bond over a number of yea...
1,b'The Godfather',b'1972',b'9.2',b'Francis Ford Coppola',"b'Marlon Brando, Al Pacino'",b'https://www.imdb.com/title/tt0068646/?pf_rd_...,NaN,"b'$6,000,000'",b'The aging patriarch of an organized crime dy...
2,b'The Godfather: Part II',b'1974',b'9.0',b'Francis Ford Coppola',"b'Al Pacino, Robert De Niro'",b'https://www.imdb.com/title/tt0071562/?pf_rd_...,NaN,"b'$13,000,000'",b'The early life and career of Vito Corleone i...
3,b'The Dark Knight',b'2008',b'9.0',b'Christopher Nolan',"b'Christian Bale, Heath Ledger'",b'https://www.imdb.com/title/tt0468569/?pf_rd_...,NaN,"b'$185,000,000'",b'When the menace known as the Joker emerges f...


In [27]:
new_df.to_csv('./imdb1.csv', index=False)
movie_data1 = pd.read_csv('imdb1.csv')
movie_data1.head(n=3)

,Title,Year,Rating,Director,Actors,URL,GENRE,Collection,Summary
0,b'The Shawshank Redemption',b'1994',b'9.2',b'Frank Darabont',"b'Tim Robbins, Morgan Freeman'",b'https://www.imdb.com/title/tt0111161/?pf_rd_...,NaN,"b'$25,000,000'",b'Two imprisoned men bond over a number of yea...
1,b'The Godfather',b'1972',b'9.2',b'Francis Ford Coppola',"b'Marlon Brando, Al Pacino'",b'https://www.imdb.com/title/tt0068646/?pf_rd_...,NaN,"b'$6,000,000'",b'The aging patriarch of an organized crime dy...
2,b'The Godfather: Part II',b'1974',b'9.0',b'Francis Ford Coppola',"b'Al Pacino, Robert De Niro'",b'https://www.imdb.com/title/tt0071562/?pf_rd_...,NaN,"b'$13,000,000'",b'The early life and career of Vito Corleone i...
